# Markov Chain Ensemble Using Recom

@author: taiyyoson

### Using 2018 Election Data

In [1]:
import pandas as pd
import geopandas as gpd
import maup
import matplotlib.pyplot as plt
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.tree import bipartition_tree
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial

import time
import os

In [3]:
## making graph
sc_graph = Graph.from_file("./SC/SC.shp")
sc_gdf = gpd.read_file("./SC/SC.shp")
sc_from_gpd_graph = Graph.from_geodataframe(sc_gdf, adjacency="rook") ## creating graph from geodataframe